In [1]:
import pandas as pd

In [5]:
lines = []
with open('/mnt/c/users/kakh/Downloads/TeamOpptyAssignment_wQualifier2.') as msx_file:
    for i in range(10):
        lines.append(msx_file.readline())

In [6]:
lines

['\ufeffAlias\tCRMOpportunityID\tPersonnelNumber\tStandardTitle\tQualifier2\tIsOpportunityOwner\tOwnershipFlag\tOpportunityCreatedDate\tOpportunityCloseDate\tOpportunityCloseYear\tOpportunityDueDate\tOpportunityDueYear\tMSXStatus\tOpportunityCloseReason\n',
 'AEGGERER\t7-NJKE6UWOM\t151275\tBusiness Program Manager\tN/A\t1\tYes\t05/15/2017\t05/14/2217\tNULL\tNULL\tN/A \tLost\tDuplicate Opportunity\n',
 'A-COBALA\t1-16RJO1X\t718150\tXR - Account Management\tN/A\t1\tYes\t08/24/2014\t12/29/2010\tFY11\t12/17/2010\tFY11\tWon\tRelationship\n',
 'CHRISHEA\t6-FGRSQ7Q7O\t803510\tAccount Executive\tFinancial Services\t0\tNo\t07/21/2014\t07/15/2025\tFY26\t08/15/2025\tFY26\tLost\tCustomer Cancelled\n',
 'AAALBREC\t6-GM25YOPWT\t513929\tAccount Executive\tManufacturing\t1\tYes\t12/16/2014\t12/28/2015\tFY16\t12/28/2015\tFY16\tWon\tRelationship\n',
 'AGCARR\t1-10BT1D5\t436096\tCloud Specialist\tN/A\t1\tYes\t03/03/2014\t05/18/2010\tFY10\t05/27/2010\tFY10\tWon\tRelationship\n',
 'AKONCKES\t7-QQDEN4JPC\t1

In [10]:
msx_data = pd.read_csv('/mnt/c/users/kakh/Downloads/msx_data.tsv', sep='\t')

In [11]:
msx_data.head()

,Alias,CRMOpportunityID,PersonnelNumber,StandardTitle,Qualifier2,IsOpportunityOwner,OwnershipFlag,OpportunityCreatedDate,OpportunityCloseDate,OpportunityCloseYear,OpportunityDueDate,OpportunityDueYear,MSXStatus,OpportunityCloseReason
0,AEGGERER,7-NJKE6UWOM,151275,Business Program Manager,NaN,1,Yes,05/15/2017,05/14/2217,NaN,NaN,N/A,Lost,Duplicate Opportunity
1,A-COBALA,1-16RJO1X,718150,XR - Account Management,NaN,1,Yes,08/24/2014,12/29/2010,FY11,12/17/2010,FY11,Won,Relationship
2,CHRISHEA,6-FGRSQ7Q7O,803510,Account Executive,Financial Services,0,No,07/21/2014,07/15/2025,FY26,08/15/2025,FY26,Lost,Customer Cancelled
3,AAALBREC,6-GM25YOPWT,513929,Account Executive,Manufacturing,1,Yes,12/16/2014,12/28/2015,FY16,12/28/2015,FY16,Won,Relationship
4,AGCARR,1-10BT1D5,436096,Cloud Specialist,NaN,1,Yes,03/03/2014,05/18/2010,FY10,05/27/2010,FY10,Won,Relationship
